### Naukri Web Scraper

Ajay Kasturirangan
02-July-2022

#### Import Required Libraries

In [1]:
import requests
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import math

#### Input

In [2]:
Job = "Data Analyst"
Location = "Chennai"

#### Identify Data Source

In [3]:
#url = 'https://www.naukri.com/' + Job.replace(" ","-") + '-jobs-in-' + Location + '?k=' + Job.replace(" ","%20") + '&l=bangalore'
#url
url = 'https://www.naukri.com/'+ Job.replace(" ","-") + '-jobs-in-' + Location
url

'https://www.naukri.com/Data-Analyst-jobs-in-Chennai'

#### Importing Data

In [4]:
def import_data(url):
    page = requests.get(url)
    driver = webdriver.Chrome('E:\\chromedriver_win32 (1)\\chromedriver.exe')
    driver.get(url)
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source,'html5lib')
    driver.close()
    return soup

#### Page Counter

In [5]:
def page_counter(soup):
    results = soup.find(class_="sortAndH1Cont")
    display = results.find_all('span',class_="fleft grey-text mr-5 fs12")
    return math.ceil(int(display[0].text.split("of ")[1])/20)

#### Extraxt Info

In [6]:
def extract_info(postings,soup):
    results = soup.find(class_='list')
    jobs = results.find_all('article',class_='jobTuple bgWhite br4 mb-8')
    for job in jobs:
        URL = job.find('a',class_ = "title fw500 ellipsis").get('href')
        title = job.find('a',class_ = "title fw500 ellipsis")
        exp = job.find('span',class_ ="ellipsis fleft fs12 lh16").text
        company = job.find('a',class_="subTitle ellipsis fleft").text
        SKILLS =  job.find_all('li',class_="fleft fs12 grey-text lh16 dot")
        skills = []
        for skill in SKILLS:
            skills.append(skill.text)
        postings = postings.append({'URL':URL,'Title':title,'Exp':exp,'Company':company,'Skills':skills},ignore_index=True)
    return postings   

#### Main

In [7]:
%%time
jobsearch = pd.DataFrame(columns=['URL','Title','Exp','Company','Skills'])
soup1 = import_data(url)
pages = page_counter(soup1)
print('Total number of pages for {} in {} is {}'.format(Job,Location,pages))

<ipython-input-4-7c7641d5e5bf>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('E:\\chromedriver_win32 (1)\\chromedriver.exe')


Total number of pages for Data Analyst in Chennai is 80
Wall time: 15.3 s


Scraping only 5 pages for faster implementation

In [8]:
%%time
pages = 5
for i in range(1,pages+1):
    page_url = url+"-"+ str(i)
    page_soup = import_data(page_url)
    jobsearch = extract_info(jobsearch,page_soup)
jobsearch.to_csv('jobsearch.csv')

<ipython-input-4-7c7641d5e5bf>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('E:\\chromedriver_win32 (1)\\chromedriver.exe')


Wall time: 1min 15s


In [9]:
jobsearch.head()

,URL,Title,Exp,Company,Skills
0,https://www.naukri.com/job-listings-senior-dat...,[Senior Data Analyst],5-10 Yrs,manoramaONLINE,"[data analysis, Data Analyst, sql, python, int..."
1,https://www.naukri.com/job-listings-mis-execut...,[MIS Executive / Data Analyst - MPS Limited],2-5 Yrs,"MPS Ltd.- Guindy, Chennai","[MIS Reporting, advanced excel, Data Analyst, ..."
2,https://www.naukri.com/job-listings-business-d...,[BUSINESS DATA ANALYST - GOOGLE ANALYTICS],5-7 Yrs,Lennox India Technology Centre Private Limited,"[Google Analytics, Business Data Analyst, digi..."
3,https://www.naukri.com/job-listings-business-a...,[Business Analyst/Data analyst - Payments Domain],4-8 Yrs,CGI,"[data analysis, SWIFT, SQL queries, Functional..."
4,https://www.naukri.com/job-listings-senior-lea...,[Senior/Lead - data Analyst],5-10 Yrs,CSS Corp,"[Data Analysis, SQL, Python, verbal, analytica..."


In [10]:
jobsearch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      100 non-null    object
 1   Title    100 non-null    object
 2   Exp      100 non-null    object
 3   Company  100 non-null    object
 4   Skills   100 non-null    object
dtypes: object(5)
memory usage: 4.0+ KB
